In [1]:
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np 
import time
import pyvisa as visa

In [3]:
VISA_RM = visa.ResourceManager('@py')

In [19]:
import numpy as np 
import pyvisa as visa
import yaml 

VISA_RM = visa.ResourceManager('@py')
class RigolSupply():
    
    '''Used to control a single Rigol PSU.'''
    
    
    def __init__(self, address, n_ch, visa_resource_manager=VISA_RM):
        resource_str = f'TCPIP0::{address:s}::INSTR'
        print(f"Building {resource_str}")
        self.resource = VISA_RM.open_resource(resource_str, write_termination='\n', read_termination='\n')

        self.write = self.resource.write
        self.query = self.resource.query
        
        self.n_ch = n_ch
        
        self.voltages = np.zeros(n_ch)
        #self.currents = np.zeros(n_ch)
        
        #self.ovp = np.zeros(n_ch)
        self.ocp = np.zeros(n_ch)
        
    @property
    def IDN(self):
        return self.query("*IDN?")
    
    @property
    def IP(self):
        return self.query(":SYSTem:COMMunicate:LAN:IPADdress?")
    
    @property
    def IDENTITY(self):
        return f"IDN: {self.IDN.split(',')[-2]} IP: {self.IP}"
        
    def ask(self, question, verbose=False):
        response = self.query(question)
        if verbose:
            print("Question: {0:s} - Response: {1:s}".format(question, str(response)))
        return response
    
    def tell(self, statement):
        return self.write(statement)
    
    def enable_output(self, ch):
        return self.tell(f"OUTP:STAT CH{ch},ON")
    
    def disable_output(self, ch):
        return self.tell(f"OUTP:STAT CH{ch},OFF")
    
    
    def set_voltage(self, ch, voltage):
        self.tell(f":SOURCE{ch}:VOLT {voltage}")
        
    def get_voltage(self):
        return [self.query(f":SOURCE{ch+1}:VOLT?") for ch in range(self.n_ch)]        
    
    def set_ocp(self, ch, ocp):
        self.tell(f":OUTP:OCP:VAL CH{ch},{ocp}")
    
    def get_ocp(self):
        return [self.query(f":OUTP:OCP:VAL? CH{ch+1}") for ch in range(self.n_ch)]

    
class RigolArray():    
    
    '''
    
    Used to build and deploy an array of RigolSupply objects.
    Designed to be used with configuration files: 
    
        #Supply voltages.
        #Note: Negative voltage is interpreted in software as "do not use".
        S3:
            IP: "10.10.1.53"
            NCH: 2
            CH1:
                V: -99
                OCP: 0.01
            CH2: 
                V: 3.6
                OCP: 2.0

        S2:
            IP: "10.10.1.52"
            NCH: 3
            CH1:
                V: 5.0
                OCP: 1.25
            CH2: 
                V: -99
                OCP: 0.01
            CH3:
                V: 2.4
                OCP: 3.0        
    
    
    '''
    
    def __init__(self, psconfig_filename):
        with open(psconfig_filename) as f:
            supply_cfg = yaml.load(f, Loader=yaml.FullLoader)

        self.supply_handlers = []
        for supply in supply_cfg:
            ps = RigolSupply(supply_cfg[supply]['IP'], supply_cfg[supply]['NCH'])   
            print(ps.IDENTITY)

            for ch in range(ps.n_ch):
                ps.tell(f":OUTP:OCP CH{ch+1},ON")
                ps.voltages[ch] = supply_cfg[supply][f"CH{ch+1}"]['V']
                ps.ocp[ch] = supply_cfg[supply][f"CH{ch+1}"]['OCP']
                if ps.voltages[ch] > 0:
                    ps.set_voltage(ch+1, ps.voltages[ch])
                    ps.set_ocp(ch+1, ps.ocp[ch])
                else:
                    ps.set_voltage(ch+1, 0)
                    ps.set_ocp(ch+1, 0.001)            


            self.supply_handlers += [ps]        
    
    def apply_to_all(self, ask=None, tell=None):
        for supply in self.supply_handlers:
            if ask is not None:
                supply.ask(ask)
            if tell is not None:
                supply.tell(tell)

    def power_cycle_all_supplies(self):
        for supply in self.supply_handlers:
            for ch in range(supply.n_ch):
                print(supply.enable_output(ch+1))
                time.sleep(1)
                print(supply.disable_output(ch+1))

    def report_status(self):
        for supply in self.supply_handlers:
            print(supply.IDENTITY)
            print(f"\tChannel\t| Status\t| V\t\t| I (A)\t\t| P (W)  ")
            channel_stats = []
            channel_out_vals = []
            for ch in range(supply.n_ch):
                stat = supply.query(f"OUTP:STAT? CH{ch+1}")
                vals = supply.query(f"MEASure:ALL? CH{ch+1}")
                vals = vals.split(',')
                vals = [float(i) for i in vals]
                print(f"\t{ch+1:7d}\t| {stat:8s}\t| {vals[0]:3.4f}\t| {vals[1]:3.4f}\t| {vals[2]:3.4f}")
            print()

    def power_up_all(self, verbose=False):
        for supply in supply_handlers:
            for ch in range(supply.n_ch):
                if supply.voltages[ch] > 0:
                    supply.enable_output(ch+1)        
        if verbose:
            time.sleep(1.0)
            report_status(supply_handlers)


    def power_down_all(self, verbose=False):
        if verbose:
            report_status(supply_handlers)
            print("----------------------")
        for supply in supply_handlers:
            for ch in range(supply.n_ch):
                supply.disable_output(ch+1)
        if verbose:
            time.sleep(1.0)
            report_status(supply_handlers)

In [6]:
with open('supply_defaults.yaml') as f:
    supply_cfg = yaml.load(f, Loader=yaml.FullLoader)

In [7]:
supply_cfg

{'S3': {'IP': '10.10.1.53',
  'NCH': 2,
  'CH1': {'V': -99, 'OCP': 0.01},
  'CH2': {'V': 3.6, 'OCP': 2.0}},
 'S2': {'IP': '10.10.1.52',
  'NCH': 3,
  'CH1': {'V': 5.0, 'OCP': 1.25},
  'CH2': {'V': -99, 'OCP': 0.01},
  'CH3': {'V': 2.4, 'OCP': 3.0}},
 'S1': {'IP': '10.10.1.51',
  'NCH': 2,
  'CH1': {'V': 5.4, 'OCP': 0.3},
  'CH2': {'V': 3.4, 'OCP': 4.0}},
 'S0': {'IP': '10.10.1.50',
  'NCH': 2,
  'CH1': {'V': 5.4, 'OCP': 0.3},
  'CH2': {'V': 1.9, 'OCP': 1.5}}}

In [9]:
for supply in supply_cfg:
    print(supply, supply_cfg[supply])

S3 {'IP': '10.10.1.53', 'NCH': 2, 'CH1': {'V': -99, 'OCP': 0.01}, 'CH2': {'V': 3.6, 'OCP': 2.0}}
S2 {'IP': '10.10.1.52', 'NCH': 3, 'CH1': {'V': 5.0, 'OCP': 1.25}, 'CH2': {'V': -99, 'OCP': 0.01}, 'CH3': {'V': 2.4, 'OCP': 3.0}}
S1 {'IP': '10.10.1.51', 'NCH': 2, 'CH1': {'V': 5.4, 'OCP': 0.3}, 'CH2': {'V': 3.4, 'OCP': 4.0}}
S0 {'IP': '10.10.1.50', 'NCH': 2, 'CH1': {'V': 5.4, 'OCP': 0.3}, 'CH2': {'V': 1.9, 'OCP': 1.5}}


In [10]:
supply

'S0'

In [11]:
s0 = supply_handlers[0]

NameError: name 'supply_handlers' is not defined

In [12]:
ps = RigolSupply(supply_cfg[supply]['IP'], supply_cfg[supply]['NCH'])


Building TCPIP0::10.10.1.50::INSTR


In [13]:
report_status(supply_handlers)

NameError: name 'supply_handlers' is not defined

In [ ]:
def power_up_all(supply_handlers, verbose=False):
    for supply in supply_handlers:
        for ch in range(supply.n_ch):
            supply.enable_output(ch+1)        
    if verbose:
        report_status(supply_handlers)

In [ ]:
supply = supply_handlers[0]

In [15]:
psconfig_filename = 'supply_defaults.yaml'

In [18]:
with open(psconfig_filename) as f:
    supply_cfg = yaml.load(f, Loader=yaml.FullLoader)

supply_handlers = []
for supply in supply_cfg:
    ps = RigolSupply(supply_cfg[supply]['IP'], supply_cfg[supply]['NCH'])   
    print(ps.IDENTITY)
    
    for ch in range(ps.n_ch):
        ps.tell(f":OUTP:OCP CH{ch+1},ON")
        ps.voltages[ch] = supply_cfg[supply][f"CH{ch+1}"]['V']
        ps.ocp[ch] = supply_cfg[supply][f"CH{ch+1}"]['OCP']
        if ps.voltages[ch] > 0:
            ps.set_voltage(ch+1, ps.voltages[ch])
            ps.set_ocp(ch+1, ps.ocp[ch])
        else:
            ps.set_voltage(ch+1, 0)
            ps.set_ocp(ch+1, 0.001)            
            

    supply_handlers += [ps]
    
for supply in supply_handlers:
    print(supply.get_ocp())
    print(supply.get_voltage())

Building TCPIP0::10.10.1.53::INSTR
IDN: DP8E214200286 IP: 10.10.1.53
Building TCPIP0::10.10.1.52::INSTR
IDN: DP8B212500667 IP: 10.10.1.52
Building TCPIP0::10.10.1.51::INSTR
IDN: DP8E203700164 IP: 10.10.1.51
Building TCPIP0::10.10.1.50::INSTR
IDN: DP8E203100159 IP: 10.10.1.50
['0.0010', '2.000']
['0.000', '3.600']
['1.250', '0.001', '3.000']
['5.000', '0.000', '2.400']
['0.3000', '4.000']
['5.400', '3.400']
['0.3000', '1.500']
['5.400', '1.900']


In [ ]:
report_status(supply_handlers)

In [ ]:
power_up_all(supply_handlers, True)

In [ ]:
power_down_all(supply_handlers, True)

In [21]:
supplies = RigolArray('supply_defaults.yaml')

Building TCPIP0::10.10.1.53::INSTR
IDN: DP8E214200286 IP: 10.10.1.53
Building TCPIP0::10.10.1.52::INSTR
IDN: DP8B212500667 IP: 10.10.1.52
Building TCPIP0::10.10.1.51::INSTR
IDN: DP8E203700164 IP: 10.10.1.51
Building TCPIP0::10.10.1.50::INSTR
IDN: DP8E203100159 IP: 10.10.1.50


In [22]:
supplies.report_status()

IDN: DP8E214200286 IP: 10.10.1.53
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| OFF     	| 0.0000	| 0.0000	| 0.0000
	      2	| OFF     	| 0.0000	| 0.0000	| 0.0000

IDN: DP8B212500667 IP: 10.10.1.52
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| OFF     	| 0.0000	| 0.0000	| 0.0000
	      2	| OFF     	| 0.0000	| 0.0000	| 0.0000
	      3	| OFF     	| 0.0000	| 0.0000	| 0.0000

IDN: DP8E203700164 IP: 10.10.1.51
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| OFF     	| 0.0000	| 0.0000	| 0.0000
	      2	| OFF     	| 0.0000	| 0.0000	| 0.0000

IDN: DP8E203100159 IP: 10.10.1.50
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| OFF     	| 0.0000	| 0.0000	| 0.0000
	      2	| OFF     	| 0.0000	| 0.0000	| 0.0000



In [23]:
supplies.power_up_all(True)

IDN: DP8E214200286 IP: 10.10.1.53
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| OFF     	| 0.0000	| 0.0000	| 0.0000
	      2	| ON      	| 3.6010	| 0.0000	| 0.0000

IDN: DP8B212500667 IP: 10.10.1.52
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 5.0014	| 0.0000	| 0.0000
	      2	| OFF     	| 0.0000	| 0.0000	| 0.0000
	      3	| ON      	| 2.4009	| 0.0000	| 0.0000

IDN: DP8E203700164 IP: 10.10.1.51
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 5.4020	| 0.0013	| 0.0070
	      2	| ON      	| 3.3980	| 1.0520	| 3.5750

IDN: DP8E203100159 IP: 10.10.1.50
	Channel	| Status	| V		| I (A)		| P (W)  
	      1	| ON      	| 5.4100	| 0.0025	| 0.0140
	      2	| ON      	| 1.9000	| 0.0000	| 0.0000

